In [ ]:
%load_ext autoreload

In [ ]:
from pathlib import Path
import numpy as np
from skimage import io as skio
import matplotlib.pyplot as plt
%matplotlib notebook
import flammkuchen as fl
import json

from split_dataset import SplitDataset
from fimpy.pipeline.alignment import align_2p_volume
from fimpy.pipeline.roi_extraction import correlation_map
%autoreload

In [ ]:
import tifffile as tiff
from tqdm import tqdm
from shutil import copy
from fimpy.pipeline.general import dff

In [ ]:
def restructure_for_suite2p(path, ds_name="original"):
    ds_path = path / ds_name

    suite_2p_dir = (path / "suite2p")
    suite_2p_dir.mkdir(exist_ok=True)
    for f in tqdm(list(ds_path.glob("*.h5"))):
        dest_dir = suite_2p_dir / f.stem
        dest_dir.mkdir(exist_ok=True)
        copy(f,dest_dir / f.name)

In [ ]:
source_dir = "anatomy\original"
source_dir = "original"


frames_per_plane = None
aligned_dir = "aligned"
force_realign = False
force_remake_tifs = False
force_remake_corr = False

In [ ]:
master = Path(r"\\FUNES\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\habenula\fixed")

all_fish = list(master.glob("*_f*"))
fish_dir = all_fish[-1]
print(all_fish)

In [ ]:
force_realign = False
for f in all_fish:
    print(f)
    try:
        if not(f / 'aligned').exists() or force_realign:
            align_2p_volume(SplitDataset(f / source_dir), output_dir=str(f), across_planes=False)
        else:
            print("Already aligned")

        if not(f / 'anatomy.tif').exists() or force_remake_tifs:
            ds = SplitDataset(f / "aligned")
            stack = ds[:, :, :, :]
            mean = stack.mean(0)
            print(np.shape(mean))
            tiff.imsave(str(f / "anatomy.tif"), mean)
            mean = 0
            stack = 0
        else:
            print("Already anatomized")
    except:
        print("Error during alignment or anatomy")

In [ ]:
for f in all_fish:
    if not (f / "suite2p").exists():
        print(f)
        restructure_for_suite2p(f)